# A photodissociated SPH/SPD-based magnetometer

This notebook solves the master equation for the following problem for hydrogen and deuterium:

HCl or DCl is photodissociated by circularly polarized light propagating along the $\hat{x}$-axis or $\hat{z}$-axis.
The produced H or D are assumed to have their electron spins oriented either along $+\hat{x}$ or $+\hat{z}$, respectively (the "-" case is, for the opposite absorbed photon helicity, is essentially the same, so I only look at the "+" case here), while their nuclear spins are unaffected at such short timescales and are assumed to possess random orientation. The produced state is allowed to evolve in time, and also interact with two, constant or time-dependent - B-fields oriented along the $\hat{x}$-axis and $\hat{z}$-axis.

The whole process takes place inside a tiny coil, the axis of which is along the beam-propagation-axis.
The spin precession and time evolution, thus, causes a time dependent variation of the magnetic flux inside the coil, which is picked up by the coil and measured as a voltage on an oscilloscope. So, the picked-up voltage is proportional to the time derivative of the expectation value of the appropriate electron operator, $\hat{s}_x$ or $\hat{s}_z$, which is in turn dependent on the magnetic field.

We investigate the performance of this system as a magnetometer

_Note: As things are defined here, times should be given in μs and frequencies in MHz_

In [1]:
# This is for Jupyter notebook to display cell at full viewport width
# display("text/html", "<style>.container { width:100% !important; }</style>")

In [1]:
# using QuantumOptics, PlotlyJS, ORCA, Parameters, ZChop, FFTW, Findpeaks

# include the base code containing all definitions
include("SPBase.jl");

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-17285095395524088071\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6525711080212298706\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-17285095395524088071\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

### Get help

1) Look at the comments inside SPBase.jl   
2) try `?function` for help on selected functions (those for which docstrings are defined in the SPBase.jl)

In [2]:
?ρMEt

"ρMEt" can be typed by \rho<tab>MEt

search: ρMEt



```
    ρMEt( Mandatory args: a::Atom, 
           Optional args: T::Time Array to solve for (1D Real) [default: đ.T];
         Named arguments: dirP::Symbol atomic polarization orientation [default: đ.dP | generally :x, :y, :z] 
                          γe::electron dephasing [default: đ.γe], 
                          γn::nuclear dephasing [default: đ.γn])
```

Calculates a density matrix which solves the master equation, for each time point in the times array, `T`.

The initial `ρo` used in the calculations is dictated by the initial atomic polarization argument, `dirP`, and is calculated by the `ρo` function.

The `Bx` and `Bz` magnetic field constructs defined earlier are used implicitly in the calculations.

Also, `BPx` and `BPz` are defined for pulsed magnetic fields. In the defaults struct, đ, two booleans determine if we have continuous (`Bx`, `Bz` >> `đ.withContB`) and/or pulsed (`BPx`, `BPz` >> `đ.withPulsedB`) fields. If we want the calculations to be performed for different fields than the ones defined, we have to mutate the `Bx` and/or `Bz` (and/or `BPx`, `BPz`) beforehand, e.g.:

```
    Bz.Bo = 10G
    Bz.f = 12MHz
    res = ρMEt(H)
```

If `Bz.f` and `Bx.f` are both zero, and `đ.withPulsedB == false`, then the Hamiltonian is time independent, and the regular solver (`timeevolution.master`) is used for the solution.

If `Bz.f` or `Bx.f` is non-zero, or `đ.withPulsedB == true`, then the magnetic field is time dependent, `Bo ̂e cos(2π fB t)`, where ê the direction of the field,  or `Bo ̂e exp(-(t-to)²/τ²)`, in the case of a - Gaussian - pulsed B-field. In this case the Hamiltonian is time-dependent, and the dynamic solver is used (`timeevolution.master_dynamic`).

*Notes:*

1. `Bz.Bo` and `Bz.Boff` (DC offset) can be given in Gauss, as e.g. `200G` (`G` is defined as `1E-4`)
2. The same for the `Bx`, `BPx`, and `BPz` constructs
3. Default values are in the struct `đ` (\dj), and can be changed as, e.g.: `đ.fB = 100kHz`  or  `đ.dP = :z`

***Returns:***

A list of QuantumOptics.jl density matrices solving the master equation for each point of the time array `T`.

(so, the density matrix at index `i` in the resulting list, is the solution of the master equation at time `T[i]`)


In [3]:
?solveME

search: solveME



```
    solveME(Mandatory args: a::Atom, 
             Optional args: T::Array of times to solve for (1D Real) [default: đ.T];
           Named arguments: Op_el the electronic operator to solve for (expectation value >> dS/dt >> Fourier) [default: sx]
                            dirP::Symbol :x, :y, :z atomic polarization orientation [default: đ.dP]
                            γe::electron dephasing [default: đ.γe], 
                            γn::nuclear dephasing [default: đ.γn])
```

Solves the master eq and returns the FFT of the `d<Op_el>/dt` (tupled with the list of frequencies).

*Note:* Internally, the `ρMEt` function is called, which uses the pre-defined magnetic fields `Bx` and/or `Bz` and/or `BPx` and/or `BPz`. (dependent on the values of the controlling booleans `đ.withContB` and `đ.withPulsedB`) 

***Returns:*** a tuple (frequencies list, Fourier list), both 1D arrays of Float64, `Array{Float64, 1}`


### Demo of defined functions

In [4]:
đ.T = [0.0:0.2ns:2μs;]

đ.withContB = false
Bz.Bo = 100G
Bx.Bo = 0G
Bz.f = 20MHz
Bx.f = 0MHz
Bz.θ = 0deg
Bz.Boff = 0.0 * Bz.Bo

đ.withPulsedB = true
BPx.Bo = 100G
BPx.to = 100ns
BPz.Bo = 0G


đ.dP = :z
đ.γe = 1MHz
opel = sz

ρt = ρMEt(H)                      # this gives the time-evolved ρ(t) for Hydrogen, and for default values for B, T, γe, γn
st = S_t(H, ρt, opel)                 # electron <Sx> vs t for the above calculated ρ(t)
dsdt = dSdt(st)              # time derivative of the <Sx> array
ftmp, fftmp = fourier(dsdt);  # ... and its Fourier (ftmp is the frequency list, fftmp is the Fourier)
tmpks = getPeaks(H; Op_el = opel)               # this gets the peaks and produces the following output:

Bz = Bz_Default;
Bx = Bx_Default;

BPz = BPz_Default;
BPx = BPx_Default;

tmpks

Results for current parameters:
Operator 		: sz 
AtomicPolarization 	: z 
HadContinuousB 	: false 
HadPulsedB 		: true 
BPz 			: Bpulse
  Bo: Float64 0.0
  to: Float64 0.0
  τ: Float64 0.2
  axis: Symbol z
  θ: Float64 0.0
  Boff: Float64 0.0
 
BPx 			: Bpulse
  Bo: Float64 0.01
  to: Float64 0.1
  τ: Float64 0.2
  axis: Symbol x
  θ: Float64 0.0
  Boff: Float64 0.0
 
PeakInds 		: [2842, 2847, 2836, 2982, 2715, 2852, 2831, 2967, 2857, 2826, 2730, 2862, 2955, 2821, 2867, 2873, 2945, 2816, 2741, 2879, 2810, 2936, 2885, 2927, 2751, 2905, 2912, 2919, 2760, 126] 


PeakFreqs 		: [1420.784156831366, 1423.2846569313863, 1417.7835567113423, 1490.7981596319264, 1357.2714542908582, 1425.7851570314062, 1415.2830566113223, 1483.2966593318663, 1428.2856571314262, 1412.7825565113023, 1364.7729545909183, 1430.7861572314462, 1477.2954590918182, 1410.2820564112822, 1433.2866573314661, 1436.2872574514904, 1472.2944588917783, 1407.7815563112622, 1370.274054810962, 1439.2878575715142, 1404.7809561912384, 1467.7935587117424, 1442.2884576915383, 1463.2926585317064, 1375.2750550110022, 1452.2904580916181, 1455.7911582316462, 1459.2918583716744, 1379.7759551910383, 62.5125025005001] 
PeakAmps 		: [2.5122106229531476e6, 282723.60088658746, 253329.13183251544, 252456.89808123998, 234645.79308164623, 205202.51351778503, 181961.16244842092, 174843.66307955483, 167344.64441880697, 150362.68637566574, 148372.20146845616, 146122.17923499833, 138775.89223979035, 133006.08322509195, 130416.13545536966, 122052.64136817571, 119772.47610433027, 118496.58669659197, 116226.27333

In [5]:
t_plot = Plot(đ.T, st, name = "<S>", Layout(xaxis_title = "Time (μs)"))
dt_plot = Plot(đ.T, dsdt, name = "d<S>/dt", Layout(xaxis_title = "Time (μs)"))
fft_plot = Plot(ftmp/1000, fftmp, name = "FFT(d<S>/dt)", Layout(xaxis_title = "Frequency (GHz)"))
sctmp=scatterPeaks(tmpks)
addtraces!(fft_plot, 2, sctmp)


opts = Dict(:scrollZoom => true, :toImageButtonOptions => Dict(:format => "svg", :scale => 1))
plot([t_plot ; dt_plot ; fft_plot], options = opts)

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields marker, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, xaxis1, xaxis2, xaxis3, yaxis1, yaxis2, and yaxis3"

## Results for a series of magnetic fields BELOW NOT FINISHED YET

### Hydrogen

In [10]:
# Now solve for various magnetic fields
# first define the B list to solve for | variable resolution (high-res at low fields, lower-res at higher fields)
Blist = vcat(0.0:0.00001:0.0001, 0.0002:0.0001:0.001, 0.002:0.001:0.05) 
#Blist = 0.0:0.000001:0.0001 # this Blist for the low B-field detail
# then initialize two lists to store the frequencies and amplitudes of the peaks
peakFreqs = rand(length(Blist), 4)
peakFreqsL = rand(length(Blist), 4)
peakVals = rand(length(Blist), 4)
peakValsL = rand(length(Blist), 4)
# reset the magnetic fields to default values
Bz = Bz_Default
Bx = Bx_Default
# and, finally, go through the B list
@time for (ind, b) in enumerate(Blist)
    Bz.Bo = b
    res = getPeaks(H)
    peakFreqs[ind,:] = res.PeakFreqs
    peakFreqsL[ind,:] = res.PeakFreqsLorentz
    peakVals[ind,:] = res.PeakAmps
    peakValsL[ind,:] = res.PeakAmpsLorentz
end

Bz = Bz_Default;

 11.042672 seconds (1.46 M allocations: 371.508 MiB, 0.79% gc time)


In [11]:
freqPlotH = Plot(Blist/G, peakFreqsL/GHz, mode="lines", Layout(xaxis_title = "B (G)", yaxis_title = "Peak Frequencies (GHz)", showlegend=false))

opts = Dict(:toImageButtonOptions => Dict(:format => "svg", :scale => 1))
plot(freqPlotH, options = opts)

data: [
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, showlegend, xaxis, and yaxis"

### Deuterium

In [12]:
# Now solve for various magnetic fields
# first define the B list to solve for | variable resolution (high-res at low fields, lower-res at higher fields)
Blist = vcat(0.0:0.00001:0.0001, 0.0002:0.0001:0.001, 0.002:0.001:0.05) 
# then initialize two lists to store the frequencies and amplitudes of the peaks
peakFreqsD = rand(length(Blist), 6)
peakFreqsDL = rand(length(Blist), 6)
peakValsD = rand(length(Blist), 6)
peakValsDL = rand(length(Blist), 6)
# reset the magnetic fields to default values
Bz = Bz_Default
Bx = Bx_Default
# and, finally, go through the B list
@time for (ind, b) in enumerate(Blist)
    Bz.Bo = b
    res = getPeaks(D)
    peakFreqsD[ind,:] = res.PeakFreqs
    peakFreqsDL[ind,:] = res.PeakFreqsLorentz
    peakValsD[ind,:] = res.PeakAmps
    peakValsDL[ind,:] = res.PeakAmpsLorentz
end

Bz = Bz_Default;

 17.213023 seconds (12.99 M allocations: 1.189 GiB, 2.42% gc time)


In [13]:
freqPlotD = Plot(Blist/G, peakFreqsDL/GHz, mode="lines", Layout(xaxis_title = "B (G)", yaxis_title = "Peak Frequencies (GHz)", showlegend=false))

opts = Dict(:toImageButtonOptions => Dict(:format => "svg", :scale => 1))
plot(freqPlotD, options = opts)

data: [
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y",
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, showlegend, xaxis, and yaxis"